# Perceptron Learning Algorithm

## 1.Homeworks

Learning from data [第一周作业](http://www.work.caltech.edu/homework/hw1.pdf) 的问题7-10部分, 使用感知器学习算法, 加深理解.

要知道, 现在我们是在写程序完成作业. 这个小程序就是告诉计算机, 让它按我们的想法(程序)完成任务.

题目简要, 每次试验

- 数据集 D
    - 输入空间 X: [-1, 1] x [-1, 1] 均匀分布产生.
        - 每个输入样本特征 2个
        - 样本数目为 N(下面的题目会给定)
- Target function: f
    - 随机生成一条线, 随机从 [-1, 1] x [-1, 1] 空间取两个点, 通过这两个随机点的直线作为 target function.
    -  f 一边的样本划分为 +1 类别, 在另一边的样本划分为 -1 类别. 自己设定.
    - 数据集 D 中的样本 x_n 进行 类别划分. (这样就生成完成数据集 (X, Y))
- PLA: 感知器算法学习
    - 利用PLA 得到 final function g
    - 初始权重: w(0) 为 0. 起始时, 所有样本都是被错误分类的. 因为 sign(0) = 0 
- 问题关注点:
    - 多少次迭代之后, PLA 收敛
    - f 与 g 之间的误差率. P[f(x) != g(x)], 训练集训练, 测试集测试, 才有这个误差率
 
试验次数 1000 次, 然后去平均值.

感知器算法 LFD 笔记: [LFD第三章笔记:线性模型 · Anifacc](https://anifacc.github.io/machinelearning/learningfromdata/2017/09/14/lfd-ch03-linear-model/)

PLA 的流程:

```
# PLA

1: Initialize at step t = 0 to w(0).

2: for t = 0, 1, 2, ... do
3:    the weight vector is w(t)
4:    From training dataset(x_1, y_1), ...(x_N, y_N) pick any misclassified example.
            Call the misclassified example (x*, y*)
            sign(w(t)^T x*) != y*;

5:    Update the weight: w(t+1) = w(t) + y*x*
6:    t = t+1   
```

In [1]:
import numpy as np
import random

# 生成数据集
def generate_dataset(data_size):
    # list of lists
    dataset = [[1., random.uniform(-1, 1), random.uniform(-1, 1)]
                for i in range(data_size)]
    return dataset

In [2]:
# 目标函数值, 真实值
def target_function(feature, piont_1, point_2): 
    # 样本特征:[1, x1, x2]
    x, y = feature[1], feature[2]

    x1, y1 = point_1
    x2, y2 = point_2
    slope = (y2- y1) / (x2 - x1)
    # 过两点的直线为目标函数
    # 在直线上的样本标记+1, 否则为 -1
    return 1 if y > (slope * (x - x1) + y1) else -1

In [3]:
# 假设集判断
def hypothesis(feature, weights):
    return np.sign(np.dot(feature, weights))

In [4]:
# 模型训练
def train(training_set, point_1, point_2, weights=None):
    misclassified = []
    iterations = 0
    
    if weights is None:
        weights = np.array([[0., 0., 0.]]).T
    else:
        weights = weights
    
    while True:
        for feature in training_set:
            true_label = target_function(feature, point_1, point_2)
            
            # 尚未正确分类的样本判断
            if hypothesis(feature, weights) != true_label:
                misclassified.append((feature, true_label))
        
        if not misclassified:
            break
        else:
            iterations += 1
            # 随机从错误分类的样本中选择一个样本, 进行权重更新
            x, y = random.choice(misclassified)
            # update w(t+1) = w(t) + y*x*
            adapt = np.array([x]).T * y
            weights += adapt
            misclassified = []
    
    return iterations, weights

In [5]:
# 测试集测试
def test(testing_set, point_1, point_2, weights):
    mismatches = 0
    
    for feature in testing_set:
        if hypothesis(feature, weights) != target_function(feature, point_1, point_2):
            mismatches += 1
            
    mismatches /= float(len(testing_set))
    
    return mismatches

In [6]:
# 试验1000次
num = 1000

training_size = 10
testing_size = 10

avg_iter = 0
avg_err = 0

for i in range(num):
    point_1 = (random.uniform(-1, 1), random.uniform(-1, 1))
    point_2 = (random.uniform(-1, 1), random.uniform(-1, 1))
    
    training_set = generate_dataset(training_size)
    testing_set = generate_dataset(testing_size)
    
    iterations, final_weights = train(training_set, point_1, point_2)
    avg_iter += iterations
    
    test_err = test(testing_set, point_1, point_2, final_weights)
    avg_err += test_err
    
avg_iter /= float(num) 
avg_err /= float(num)  

print("Dataset(N={0}): Average iterations:{1}(average error: {2}),".format(
            training_size, avg_iter, avg_err))

Dataset(N=10): Average iterations:9.423(average error: 0.1083),


In [7]:
def run_experiment(data_size, run_number, weights=None):
    training_size = testing_size = data_size
    
    avg_iter = 0
    avg_err = 0
    
    for i in xrange(run_number):
        # 每一次试验, 都要随机选择点, 构造目标函数 
        # 这里一定要注意, 我在这里就 debug 了好久
        point_1 = (random.uniform(-1, 1), random.uniform(-1, 1))
        point_2 = (random.uniform(-1, 1), random.uniform(-1, 1))
    
        training_set = generate_dataset(training_size)
        testing_set = generate_dataset(testing_size)
        
        
        iterations, final_weights = train(training_set, point_1, point_2)
        avg_iter += iterations
        
        test_err = test(testing_set, point_1, point_2, final_weights)
        avg_err += test_err
        
    avg_iter /= float(run_number) 
    avg_err /= float(run_number)
        
    return avg_iter, avg_err

In [8]:
if __name__ == "__main__":
    data_size = [10, 100]
    run_number = 1000
    
    for i in data_size:
        avg_iter, avg_err = run_experiment(i, run_number)
        print("Dataset(N={0}): Average iterations:{1}(average error: {2}),".format(
            i, avg_iter, avg_err))

Dataset(N=10): Average iterations:7.583(average error: 0.1094),
Dataset(N=100): Average iterations:100.401(average error: 0.01432),


# 问题:

我将上述代码直接放在 python 脚本中, 运行, 就出错.

```
# -*- coding: utf-8 -*-

import numpy as np
import random

# 生成数据集
def generate_dataset(data_size):
    # list of lists
    dataset = [[1., random.uniform(-1, 1), random.uniform(-1, 1)]
                for i in range(data_size)]
    return dataset

# 目标函数值, 真实值
def target_function(feature, piont_1, point_2):
    # 样本特征:[1, x1, x2]
    x, y = feature[1], feature[2]

    x1, y1 = point_1
    x2, y2 = point_2
    slope = (y2- y1) / (x2 - x1)
    # 过两点的直线为目标函数
    # 在直线上的样本标记+1, 否则为 -1
    return 1 if y > (slope * (x - x1) + y1) else -1

# 假设集判断
def hypothesis(feature, weights):
    return np.sign(np.dot(feature, weights))

# 模型训练
def train(training_set, point_1, point_2, weights=None):
    misclassified = []
    iterations = 0

    if weights is None:
        weights = np.array([[0., 0., 0.]]).T
    else:
        weights = weights

    while True:
        for feature in training_set:
            true_label = target_function(feature, point_1, point_2)

            # 尚未正确分类的样本判断
            if hypothesis(feature, weights) != true_label:
                misclassified.append((feature, true_label))

        if not misclassified:
            break
        else:
            iterations += 1
            # 随机从错误分类的样本中选择一个样本, 进行权重更新
            x, y = random.choice(misclassified)
            # update w(t+1) = w(t) + y*x*
            adapt = np.array([x]).T * y
            weights += adapt
            misclassified = []

    return iterations, weights

# 测试集测试
def test(testing_set, point_1, point_2, weights):
    mismatches = 0

    for feature in testing_set:
        if hypothesis(feature, weights) != target_function(feature, point_1, point_2):
            mismatches += 1

    mismatches /= float(len(testing_set))

    return mismatches

def run_experiment(data_size, run_number, weights=None):
    training_size = testing_size = data_size

    avg_iter = 0
    avg_err = 0

    for i in range(run_number):
        point_1 = (random.uniform(-1, 1), random.uniform(-1, 1))
        point_2 = (random.uniform(-1, 1), random.uniform(-1, 1))

        training_set = generate_dataset(training_size)
        testing_set = generate_dataset(testing_size)

        iterations, final_weights = train(training_set, point_1, point_2)
        avg_iter += iterations

        test_err = test(testing_set, point_1, point_2, final_weights)
        avg_err += test_err

    avg_iter /= float(run_number)
    avg_err /= float(run_number)

    return avg_iter, avg_err

if __name__ == "__main__":
    data_size = [10, 100]
    run_number = 1000

    for i in data_size:
        avg_iter, avg_err = run_experiment(i, run_number)
        print("Dataset(N={0}): Average iterations:{1}(average error: {2}),".format(
            i, avg_iter, avg_err))

```

## 错误信息:

```
Traceback (most recent call last):
  File "C:\Users\we\Desktop\perceptron.py", line 103, in <module>
    avg_iter, avg_err = run_experiment(i, run_number)
  File "C:\Users\we\Desktop\perceptron.py", line 21, in run_experiment
    iterations, final_weights = train(training_set, point_1, point_2)
  File "C:\Users\we\Desktop\perceptron.py", line 67, in train
    true_label = target_function(feature, point_1, point_2)
  File "C:\Users\we\Desktop\perceptron.py", line 44, in target_function
    x1, y1 = point_1
NameError: global name 'point_1' is not defined
[Finished in 0.259s]
```

这个就要涉及到 python 运行顺序问题. 暂且放在一边, 再学习python的时候, 碰到解决方案, 再回来尝试一下.

## 代码参考

https://github.com/blaenk/learning-from-data
    

之后可以继续简化代码, 使用 class 来封装.

待定.